In [37]:
import os
import tqdm

import torch
import torch.nn as nn
import torchvision

import numpy as np
import sklearn.preprocessing as preprocessing

In [9]:
LOCAL_DATA_DIR = '/scr/scr-with-most-space/imagenet'

In [50]:
def load_data_loader(data_dir=LOCAL_DATA_DIR, batch_size=128):
    valdir = os.path.join(data_dir, 'val')
    return torch.utils.data.DataLoader(
        torchvision.datasets.ImageFolder(valdir, torchvision.transforms.Compose([
            torchvision.transforms.Resize(256),
            torchvision.transforms.CenterCrop(224),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])),
        batch_size=batch_size, shuffle=True,
        num_workers=4)

def get_model():
    model = torchvision.models.resnet50(num_classes=1000, pretrained=True)
    model = nn.Sequential(*list(model.children())[:-1])
    return model

In [51]:
data_loader = load_data_loader()
model = get_model()
model = model.cuda()
model = model.eval()

/u/nlp/anaconda/main/anaconda3/envs/kshen6-uee/lib/python3.8/site-packages/torch/utils/data/dataloader.py:474: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /sailhome/kshen6/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [52]:
feat = []
for idx, (x, _) in enumerate(data_loader):
    if idx > 20:
        break
    x = x.cuda()
    output = model(x)
    feat.append(output.detach().cpu().numpy())

In [53]:
concat_feat = np.concatenate(feat)
concat_feat = np.squeeze(concat_feat)
print(concat_feat.shape)

(2688, 2048)


In [54]:
# normalize
normalized_feat = preprocessing.normalize(concat_feat)

In [55]:
similarities = normalized_feat @ normalized_feat.T

In [56]:
# get the average entry of the upper triangle, avoiding duplicating
print(np.mean(similarities[np.triu_indices(len(similarities))]))

0.53008693


In [58]:
# all the biases are False
list(model.children())

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
 Sequential(
   (0): Bottleneck(
     (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (downsample): Sequential(
       (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (1): BatchNorm2d(256, eps=1e-05, momentum